In [54]:
import pandas as pd
from tqdm import tqdm

train_set = pd.read_csv('dev_set.tsv', sep='\t')
# train_set = pd.read_csv('train_set.tsv', sep='\t')
policy_corpus = pd.read_csv('policy_corpus.tsv', sep='\t')

In [55]:
name_df = pd.read_csv("../../outputs/gpt-3.5-turbo-16k-0613/dev_name.tsv", sep='\t')
category_df = pd.read_csv("../../outputs/gpt-3.5-turbo-16k-0613/dev_category.tsv", sep='\t')
passage_df = pd.read_csv("../../outputs/gpt-3.5-turbo-16k-0613/dev_passage.tsv", sep='\t')
subcategory_df = pd.read_csv("../../outputs/gpt-3.5-turbo-16k-0613/dev_subcategory.tsv", sep='\t')
title_df = pd.read_csv("../../outputs/gpt-3.5-turbo-16k-0613/dev_title.tsv", sep='\t')

In [68]:
train_set["index"] = train_set["id"]
policy_corpus["index"] = policy_corpus["id"]

In [57]:
def merge_column(df: pd.DataFrame, column_df: pd.DataFrame, column_name: str):
    column_df.rename(columns={"response": column_name}, inplace=True)
    new_df = df.merge(column_df[["index", column_name]], on='index')
    return new_df

In [58]:
name_df

,index,chinese_text,response
0,616,江苏菲沃泰纳米科技股份有限公司科创板首次公开发行股票招股说明书（申报稿）,"""Jiangsu Feiwotai Nanotechnology Co., Ltd. Sci..."
1,624,翱捷科技股份有限公司科创板首次公开发行股票招股说明书（申报稿）,"""AoJie Technology Co., Ltd. Science and Techno..."
2,653,思特威（上海）电子科技股份有限公司科创板首次公开发行股票招股说明书（申报稿）,"""CMC (Shanghai) Electronic Technology Co., Ltd..."
3,623,上海澳华内镜股份有限公司科创板首次公开发行股票招股说明书（申报稿）,"""Shanghai Aohua Endoscopy Co., Ltd. Science an..."
4,676,珠海高凌信息科技股份有限公司科创板首次公开发行股票招股说明书（申报稿）,"""Zhuhai Gaoling Information Technology Co., Lt..."
...,...,...,...
139,633,上海硅产业集团股份有限公司科创板首次公开发行股票招股说明书（注册稿）,"""Shanghai Silicon Industry Group Co., Ltd. Sci..."
140,675,广东利元亨智能装备股份有限公司科创板首次公开发行股票招股说明书（上会稿）,"""Guangdong Liyuanheng Intelligent Equipment Co..."
141,486,三生国健药业（上海）股份有限公司科创板首次公开发行股票招股说明书（注册稿）,"""San Sheng Guo Jian Pharmaceutical Co., Ltd. (..."
142,661,盛美半导体设备（上海）股份有限公司科创板首次公开发行股票招股说明书（上会稿）,"""Shengmei Semiconductor Equipment (Shanghai) C..."


In [59]:
train_set = merge_column(train_set, name_df, "name")
train_set = merge_column(train_set, category_df, "category")
train_set = merge_column(train_set, passage_df, "passage")
train_set = merge_column(train_set, subcategory_df, "subcategory")
train_set = merge_column(train_set, title_df, "title")

In [60]:
train_set_df_en = train_set[["id", "name_y", "category_y", "passage_y", "subcategory_y", "title_y", "pos_ctxs", "neg_ctxs"]].copy()


In [61]:
train_set_df_en = train_set_df_en.rename(columns={"name_y": "name", "category_y": "category", "passage_y": "passage", "subcategory_y": "subcategory", "title_y": "title"})
train_set_df_en["name"] = train_set_df_en["name"].apply(lambda x: x.replace("\"", ""))
train_set_df_en["category"] = train_set_df_en["category"].apply(lambda x: x.replace("\"", ""))
train_set_df_en["subcategory"] = train_set_df_en["subcategory"].apply(lambda x: x.replace("\"", ""))
train_set_df_en["title"] = train_set_df_en["title"].apply(lambda x: x.replace("\"", ""))

In [62]:
train_set_df_en

,id,name,category,passage,subcategory,title,pos_ctxs,neg_ctxs
0,552,"Suzhou Shengke Communication Co., Ltd. Science...",New Generation Information Technology,Shengke Communication is a leading domestic Et...,Integrated circuits and semiconductors,4. Operating situation of the issuer's main bu...,"7385,8811,1852,9626,3261,7824,4714","9734,2725,5490,3504,6563,1875,9186,2436,8491,1..."
1,553,"Everest Intelligent Equipment Co., Ltd. Scienc...",New Generation Information Technology,'EFORT industrial robot CMA spraying industria...,Integrated circuits and semiconductors,2. Basic information about the main products.,"9782,1332,6120,7953,8898,9594,6642,4906","7001,2210,957,2864,6698,437,8176,5601,8702,857..."
2,556,"Beijing Nuohexizyuan Technology Co., Ltd. Scie...",New Generation Information Technology,The company relies mainly on high-throughput s...,Big data,4. Operating situation of the company's main b...,"973,667,127,1710,6566,7755,3226,2429,2904,6434","4995,397,1985,7704,7389,8836,2964,9714,9241,3507"
3,564,"Nanjing Gao Hua Technology Co., Ltd. Science a...",New Generation Information Technology,"The company's main business is the research, d...",Integrated circuits and semiconductors,4. Operating situation of the issuer's main bu...,"9572,5573,6181,2077,672,3684","2052,7001,4493,1852,9302,6563,211,3504,9186,88..."
4,566,"Juxin Technology Co., Ltd. Science and Technol...",New Generation Information Technology,The company is a leading low-power system-leve...,Smart hardware,1. Overview of the company's main business and...,"2837,4895,5490,1158,7385,7322,9875,4611","2568,3059,3082,8202,1072,9357,6960,8493,5002,8..."
...,...,...,...,...,...,...,...,...
107,555,Shanghai BlueTech Petrochemical Environmental ...,Biopharmaceutical,The company mainly provides integrated solutio...,Biopharmaceutical,(2) Overview of the company's main products,"3959,2195,3080,7313,6933,774,3420,8948","5247,4051,2145,7510,1658,8229,3279,3042,25,202..."
108,560,"Maiwei (Shanghai) Biotechnology Co., Ltd. Scie...",Biopharmaceutical,The issuer is an innovative biopharmaceutical ...,'Bio-manufacturing',4. Operating situation of the issuer's main bu...,"1869,4212,2578,5815,973,4713,8018,9766,3074","1565,8758,2904,6434,3085,2902,2429,2530,1269,9..."
109,572,Prospectus for the Initial Public Offering of ...,Biopharmaceutical,Xianzhengda Group was registered in Shanghai i...,Biological agriculture,(1) Basic information about the main business,"2188,1408,4786,8645,9239,2669,7120,9205,8836,3953","4702,8955,5385,5443,7778,4956,5797,8861,3313,2500"
110,617,"Hangzhou Botu Biological Technology Co., Ltd. ...",Biopharmaceutical,"Since its establishment, the company has been ...",Biopharmaceutical,(1) Main business of the issuer,"7704,1676,5816,9180,9223,1705,9079,4431,5403","6090,1256,4995,8319,7755,3226,5815,397,1565,22..."


In [63]:
import os
os.makedirs("../spred_en", exist_ok=True)
train_set_df_en.to_csv("../spred_en/dev_set.tsv", sep='\t', index=False)

In [65]:
name_df = pd.read_csv("../../outputs/gpt-3.5-turbo-16k-0613/policy_name.tsv", sep='\t')
category_df = pd.read_csv("../../outputs/gpt-3.5-turbo-16k-0613/policy_category.tsv", sep='\t')
passage_df = pd.read_csv("../../outputs/gpt-3.5-turbo-16k-0613/policy_content.tsv", sep='\t')
# subcategory_df = pd.read_csv("../../outputs/gpt-3.5-turbo-16k-0613/policy_subcategory.tsv", sep='\t')
title_df = pd.read_csv("../../outputs/gpt-3.5-turbo-16k-0613/policy_titlelist.tsv", sep='\t')

In [69]:
policy_corpus = merge_column(policy_corpus, name_df, "name")
policy_corpus = merge_column(policy_corpus, category_df, "category")
policy_corpus = merge_column(policy_corpus, passage_df, "content")
# policy_corpus = merge_column(policy_corpus, subcategory_df, "subcategory")
policy_corpus = merge_column(policy_corpus, title_df, "titlelist")

In [73]:
policy_df_en = policy_corpus[["id", "name_y", "category_y", "content_y", "titlelist"]].copy()


In [78]:
# policy_df_en = policy_df_en.rename(columns={"name_y": "name", "category_y": "category", "content_y": "content"})
policy_df_en["name"] = policy_df_en["name"].apply(lambda x: x.replace("\"", ""))
policy_df_en["content"] = policy_df_en["content"].apply(lambda x: x.replace("\"", ""))
policy_df_en["category"] = policy_df_en["category"].apply(lambda x: x.replace("\"", ""))
policy_df_en["titlelist"] = policy_df_en["titlelist"].apply(lambda x: x.replace("\"", ""))

In [79]:
policy_df_en

,id,name,category,content,titlelist
0,7437,Guidance Catalogue for Key Products and Servic...,'通用' translates to 'universal' or 'general' in...,Network equipment mainly includes next-generat...,New generation information technology industry...
1,8786,Guidance Catalogue for Key Products and Servic...,'通用' translates to 'universal' or 'general' in...,Network equipment mainly includes next-generat...,New generation information technology industry...
2,7963,Guidance Catalogue for Key Products and Servic...,'通用' translates to 'universal' or 'general' in...,Network equipment mainly includes next-generat...,New generation information technology industry...
3,9128,Guidance Catalogue for Key Products and Servic...,'通用' translates to 'universal' or 'general' in...,Network equipment mainly includes next-generat...,New generation information technology industry...
4,1230,Guidance Catalogue for Key Products and Servic...,'通用' translates to 'universal' or 'general' in...,Network equipment mainly includes next-generat...,New generation information technology industry...
...,...,...,...,...,...
9997,6384,Classification of Strategic Emerging Industrie...,'通用' translates to 'universal' or 'general' in...,The new technology and innovation entrepreneur...,"Related service industry, new technology and i..."
9998,4720,Classification of Strategic Emerging Industrie...,'通用' translates to 'universal' or 'general' in...,"Inspection, testing, and certification service...","Related service industry, new technology and i..."
9999,173,Classification of Strategic Emerging Industrie...,'通用' translates to 'universal' or 'general' in...,The new materials industry mainly includes adv...,New materials industry
10000,1244,Classification of Strategic Emerging Industrie...,'通用' translates to 'universal' or 'general' in...,The new materials industry mainly includes adv...,New materials industry


In [80]:
policy_df_en.to_csv("../spred_en/policy_corpus.tsv", sep='\t', index=False)